# Template generation notebook

In this notebook we generate the yaml templates that will serve as inputs for the many instanciation of our pipeline.
All the templates are saved in the same folder (before being givne as input).
When the pipeline has finished running for a given template, this very template is saved in the output folder.

In [32]:
import os
import yaml
import numpy as np

from utils import check_folder, read_yaml, save_yaml, write

In [33]:
path_to_main = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/code/fMRI/main.py"

In [34]:
subject_dict = {'english': [57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
                    72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 91, 92, 93,
                    94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 108, 109, 110, 113, 114, 115],
                'french':[1]
               }

In [35]:
hrf_list = [
    'spm', # hrf model used in SPM
    'spm + derivative', # SPM model plus its time derivative (2 regressors)
    'spm + derivative + dispersion', # idem, plus dispersion derivative (3 regressors)
    'glover', # this one corresponds to the Glover hrf
    'glover + derivative', # the Glover hrf + time derivative (2 regressors)
    'glover + derivative + dispersion' # idem + dispersion derivative
]


In [36]:
template = {
    'tr': 2.,
    'nb_runs': 9,
    'nb_runs_test': 1,
    'subject': None,
    'scaling_mean': True,
    'scaling_var': True,
    'parallel': False,
    'cuda': True,
    'hrf': None,
    'base': 10.0,
    'voxel_wise': True,
    'atlas': 'cort-prob-2mm',
    'seed': 1111,
    'alpha_percentile': 99.9,
    'alpha': None,
    'alpha_min_log_scale': 2,
    'alpha_max_log_scale': 5,
    'nb_alphas': 10,
    'optimizing_criteria': 'R2',
    'encoding_model': 'Ridge()',
    'masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/global_masker_english",
    'smoothed_masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/smoothed_global_masker_english",
    'path_to_root': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",
    'path_to_fmridata': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/fMRI",
    'offset_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/wave/english/onsets-offsets",
    'duration_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",
    'language': None,
    'output': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/", # Path to the output folder
    'input': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/", # Path to the folder containing the representations
    'models': [],
    'model_name': None,
}

In [37]:
def get_model_template(
    model_name, 
    layers, 
    hidden_states, 
    attention_heads, 
    heads,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal,
    input_template='activations'):
    
    columns_to_retrieve = []
    if hidden_states:
        columns_to_retrieve = ['hidden_state-layer-{}-{}'.format(layer, i) for layer in layers for i in range(1, 769)]
    if attention_heads:
        columns_to_retrieve += ['attention-layer-{}-head-{}-{}'.format(layer, head, i) for layer in layers for head in heads for i in range(1, 65)]
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'input_template': input_template # cls / sep / activations
      }
    return result
    

In [40]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2_layers/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2_layers/shell_commands/"

In [41]:
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

### Determining best hrf kernel

In this part we determine the best hrf kernel to use with the Bert-base-cased model.

In [25]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/jobs.txt"

In [26]:
subject_list = [57, 63, 67, 77] # identified correct subjects
model_name = 'bert-base-cased'
hidden_layer_list = [[1], [6], [8], [9], [10], [12]]
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [27]:
for subject in subject_list:
    for hrf in hrf_list:
        template['subject'] = subject
        template['hrf'] = hrf
        hrf = hrf.replace(' ', '')
        # hidden-states analysis
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                    False, None, "Bert_hidden-layer-{}".format(hidden_layers),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_{}_hidden-layer-{}'.format(hrf, subject, model_name, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_{}_hidden-layer-{}.yml'.format(hrf, subject, model_name, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_{}_hidden-layer-{}.sh'.format(hrf, subject, model_name, hidden_layers[0])))
        
        # Attention analysis
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "Bert_attention-layer-{}_head-{}".format(layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_{}_attention-layer-{}_head-{}'.format(hrf, subject, model_name, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_{}_attention-layer-{}_head-{}.yml'.format(hrf, subject, model_name, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_{}_attention-layer-{}_head-{}.sh'.format(hrf, subject, model_name, layer, head)))
            

In [28]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

I found that the 'spm + derivative' works best but is almost identical to 'spm' alone. So, in order not to have huge design-matrix, I choose to use 'spm'.

### Analysis of Bert bucket versus Sequential

In this part we use Bert-base-cased model and fit a regression model for each layer hidden-states or attention heads for classical sentence-level prediction and sequential prediction. 

In [36]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_layers/jobs.txt"

In [37]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [38]:
model_names = ['bert-base-cased', 'gpt2', 'bert-base-cased_sequential']
hidden_layer_list = [[i] for i in range(13)]
attention_layer_list = [[i] for i in range(1, 13)]
heads = np.arange(1, 13)
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [40]:
for model_name in model_names:
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_hidden-layer-{}".format(model_name, hidden_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_hidden-layer-{}'.format(model_name, subject, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_hidden-layer-{}.yml'.format(model_name, subject, hidden_layers[0]))

            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_hidden-layer-{}.sh'.format(model_name, subject, hidden_layers[0])))
        
        # attention layers comparison
        for attention_layers in attention_layer_list:
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_attention-layer-{}".format(model_name, attention_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention_layer-{}'.format(model_name, subject, attention_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}.yml'.format(model_name, subject, attention_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}.sh'.format(model_name, subject, attention_layers[0])))
            
        # specific attention heads comparison    
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "{}_attention-layer-{}_head-{}".format(model_name, layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention-layer-{}_head-{}'.format(model_name, subject, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}_head-{}.yml'.format(model_name, subject, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}_head-{}.sh'.format(model_name, subject, layer, head)))
            

In [41]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

In [42]:
len(command_lines)

5049

### Attention heads analysis 

In this part, we look at the regression model resulting from specific attention heads.

attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]

Already included in the first comparison...

### Analysis of fine-tuned Bert models

In this part, we fit each layer hidden-states (and potentially specific heads attention) of each fine-tuned Bert model to the fMRI data of all english subjects.

### Bert VS Roberta

In [1]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/jobs.txt"

In [ ]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [ ]:
model_names = ['roberta-base']
hidden_layer_list = [[i] for i in range(13)]
attention_layer_list = [[i] for i in range(1, 13)]
heads = np.arange(1, 13)
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [ ]:
for model_name in model_names:
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_hidden-layer-{}".format(model_name, hidden_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_hidden-layer-{}'.format(model_name, subject, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_hidden-layer-{}.yml'.format(model_name, subject, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_hidden-layer-{}.sh'.format(model_name, subject, hidden_layers[0])))
        
        # attention layers comparison
        for attention_layers in attention_layer_list:
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_attention-layer-{}".format(model_name, attention_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention_layer-{}'.format(model_name, subject, attention_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}.yml'.format(model_name, subject, attention_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}.sh'.format(model_name, subject, attention_layers[0])))
            
        # specific attention heads comparison    
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "Bert_attention-layer-{}_head-{}".format(layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention-layer-{}_head-{}'.format(model_name, subject, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}_head-{}.yml'.format(model_name, subject, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}_head-{}.sh'.format(model_name, subject, layer, head)))
            

In [ ]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

In [ ]:
len(command_lines)

### Bert full

In [9]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2/jobs.txt"


In [10]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [11]:
model_names = ['gpt2_scaled']
hidden_layer_list = [[i for i in range(13)]]
attention_layer_list = [[i for i in range(1, 13)]]
heads = np.arange(1, 13)
command_lines = []
path_to_sh = []
data_compression = ['pca']
ncomponents = [300]

In [62]:
def get_test_template(
    model_name, 
    layers, 
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal,
    input_template='activations'):
    
    columns_to_retrieve = ['{}_component-{}'.format(model_name, i) for i in range(1, 301)]
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'input_template': input_template # cls / sep / activations
      }
    return result
    

In [63]:
for index, model_name in enumerate(model_names):
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model = get_test_template(model_name, hidden_layers, 
                                            "{}_{}".format(model_name, subject),
                                            data_compression[index], ncomponents[index],
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}'.format(model_name, subject)
            yaml_path = os.path.join(templates_folder, '{}_{}.yml'.format(model_name, subject))

            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}.sh'.format(model_name, subject)))
        

In [12]:
for index, model_name in enumerate(model_names):
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            name_suppelement = '_{}_{}'.format(data_compression[index], ncomponents[index]) if data_compression[index] else ''
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_all-hidden-layers{}".format(model_name, name_suppelement),
                                            data_compression[index], ncomponents[index],
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_all-hidden-layers{}'.format(model_name, subject, name_suppelement)
            yaml_path = os.path.join(templates_folder, '{}_{}_all-hidden-layers{}.yml'.format(model_name, subject, name_suppelement))

            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_all-hidden-layers{}.sh'.format(model_name, subject, name_suppelement)))
        
        # attention layers comparison
        for attention_layers in attention_layer_list:
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_all-attention-layers{}".format(model_name, name_suppelement),
                                            data_compression[index], ncomponents[index],
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_all-attention-layers{}'.format(model_name, subject, name_suppelement)
            yaml_path = os.path.join(templates_folder, '{}_{}_all-attention-layers{}.yml'.format(model_name, subject, name_suppelement))

            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_all-attention-layers{}.sh'.format(model_name, subject, name_suppelement)))
            

In [13]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

In [22]:
template

{'tr': 2.0,
 'nb_runs': 9,
 'nb_runs_test': 1,
 'subject': 115,
 'scaling_mean': True,
 'scaling_var': True,
 'parallel': False,
 'cuda': True,
 'hrf': 'spm',
 'voxel_wise': True,
 'atlas': 'cort-prob-2mm',
 'seed': 1111,
 'alpha_percentile': 99.9,
 'alpha': None,
 'alpha_min_log_scale': 2,
 'alpha_max_log_scale': 5,
 'nb_alphas': 10,
 'optimizing_criteria': 'R2',
 'encoding_model': 'Ridge()',
 'masker_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/global_masker_english',
 'smoothed_masker_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/smoothed_global_masker_english',
 'path_to_root': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/',
 'path_to_fmridata': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/fMRI',
 'offset_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/wave/english/onsets-offsets',
 'duration_path': '/neuros

### Bert controls

In [22]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_control/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_control/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_control/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

In [23]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix
seeds = [24, 213, 1111, 61, 183]

In [24]:
#model_names = ['bert-base-cased_control_{}_layer-{}'.format(seed, layer) for seed in seeds for layer in range(13)]
hidden_layer_list = [[i] for i in range(13)]
attention_layer_list = [[i] for i in range(1, 13)]
heads = np.arange(1, 13)
command_lines = []
path_to_sh = []

In [25]:
for seed in seeds:
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject

        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model_name = 'bert-base-cased_control_{}_layer-{}'.format(seed, hidden_layers[0])
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_hidden-layer-{}".format(model_name, hidden_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_hidden-layer-{}'.format(model_name, subject, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_hidden-layer-{}.yml'.format(model_name, subject, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_hidden-layer-{}.sh'.format(model_name, subject, hidden_layers[0])))

        # attention layers comparison
        for attention_layers in attention_layer_list:
            model_name = 'bert-base-cased_control_{}_layer-{}'.format(seed, attention_layers[0])
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_attention-layer-{}".format(model_name, attention_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention_layer-{}'.format(model_name, subject, attention_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}.yml'.format(model_name, subject, attention_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}.sh'.format(model_name, subject, attention_layers[0])))

In [26]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

### GPT2

In [47]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2_layers/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2_layers/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_gpt2_layers/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

In [48]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix
seeds = [24, 213, 1111, 61, 183]

In [49]:
model_names = ['gpt2_scaled']
hidden_layer_list = [[i] for i in range(13)]
attention_layer_list = [[i] for i in range(1, 13)]
heads = np.arange(1, 13)
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [50]:
for model_name in model_names:
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_hidden-layer-{}".format(model_name, hidden_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_hidden-layer-{}'.format(model_name, subject, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_hidden-layer-{}.yml'.format(model_name, subject, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_hidden-layer-{}.sh'.format(model_name, subject, hidden_layers[0])))
        
        # attention layers comparison
        for attention_layers in attention_layer_list:
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_attention-layer-{}".format(model_name, attention_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention_layer-{}'.format(model_name, subject, attention_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}.yml'.format(model_name, subject, attention_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}.sh'.format(model_name, subject, attention_layers[0])))
            
        # specific attention heads comparison    
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "Bert_attention-layer-{}_head-{}".format(layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention-layer-{}_head-{}'.format(model_name, subject, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}_head-{}.yml'.format(model_name, subject, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}_head-{}.sh'.format(model_name, subject, layer, head)))
            

In [51]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

### LSTM

In [34]:
def get_lstm_template(
    model_name, 
    layers, 
    ninp,
    nhid,
    parameters,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal,
    includ_surprisal,
    includ_entropy,
    input_template='activations'):
    
    columns_to_retrieve = []
    for param in parameters:
        columns_to_retrieve = ['{}-layer-{}-{}'.format(param, layer, i) for layer in layers for i in range(1, nhid + 1)]
    if includ_surprisal:
        columns_to_retrieve += ['surprisal']
    if includ_entropy:
        columns_to_retrieve += ['entropy']
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'input_template': input_template # activations
      }
    return result
    

In [41]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_lstm/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_lstm/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_lstm/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

In [42]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_lstm/jobs.txt"
command_lines = []
path_to_sh = []

In [43]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [49]:
model_names = ['LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_02_wiki_kristina_english',
 'LSTM_embedding-size_600_nhid_300_nlayers_1_dropout_02_wiki_kristina_english']
hidden_layer_list = [[i for i in range(1, 3)], [1]]
parameters_list = [['in'], ['forget'], ['out'], ['c_tilde'], ['cell']] #['hidden'],
data_compression = ['pca', None]
ncomponents = [300, None]
shift_surprisal = False
includ_surprisal = False
includ_entropy = False
params = [{'ninp': 650, 'nhid': 650, 'nlayers': 2}, {'ninp': 600, 'nhid': 300, 'nlayers': 1}]

In [50]:
for index, model_name in enumerate(model_names):
    param = params[index]
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        for parameters in parameters_list:
        
            # hidden layers comparison
            model = get_lstm_template(model_name, 
                                        hidden_layer_list[index], 
                                        param['ninp'],
                                        param['nhid'],
                                        parameters,
                                        "{}_all-{}-layers".format(model_name, parameters[0]),
                                        data_compression[index], 
                                        ncomponents[index],
                                        "word+punctuation",
                                        None,
                                        shift_surprisal,
                                        includ_surprisal,
                                        includ_entropy,
                                        input_template='activations')

            template['models'] = [model]
            template['model_name'] = '{}_{}_all-{}-layers'.format(model_name, subject, parameters[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_all-{}-layers.yml'.format(model_name, subject, parameters[0]))

            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_all-{}-layers.sh'.format(model_name, subject, parameters[0])))


In [51]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

### Glove

In [17]:
def get_glove_template(
    model_name, 
    embedding_size,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    input_template='activations'):
    
    columns_to_retrieve = ['embedding-{}'.format(i) for i in range(1, embedding_size + 1)]
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'input_template': input_template # activations
      }
    return result
    

In [18]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

In [19]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/jobs.txt"
command_lines = []
path_to_sh = []

In [20]:
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [21]:
model_names = ['glove_embeddings']
embedding_size = [300]
data_compression = [None]
ncomponents = [None]
shift_surprisal = False

In [22]:
for index, model_name in enumerate(model_names):
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        model = get_glove_template(model_name, 
                                    embedding_size[index],
                                    'glove_{}'.format(embedding_size[index]),
                                    data_compression[index], 
                                    ncomponents[index],
                                    "word+punctuation",
                                    None,
                                    input_template='activations')

        template['models'] = [model]
        template['model_name'] =  'glove_{}_{}'.format(embedding_size[index], subject)
        yaml_path = os.path.join(templates_folder, 'glove_{}_{}.yml'.format(embedding_size[index], subject))

        save_yaml(template, yaml_path)
        command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
        path_to_sh.append(os.path.join(sh_folder, 'glove_{}_{}.sh'.format(embedding_size[index], subject)))


In [23]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

### Basic Features

In [52]:
def get_BF_template(
    model_name, 
    column_names,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal,
    input_template='activations'):
    
    columns_to_retrieve = column_names
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'input_template': input_template # activations
      }
    return result
    

In [53]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_test/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_test/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_test/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

In [54]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_test/jobs.txt"
command_lines = []
path_to_sh = []
hrf = 'spm' # 'spm + derivative' slightly better but not so much as to double the design-matrix

In [55]:
model_names = ['wordrate']
data_compression = [None, None, None, None]
ncomponents = [None, None, None, None]
shift_surprisal = [False, False, False, False]
offset_types = ["word"] * len(model_names) # ["word", "rms_0.01"]

In [56]:
for index, model_name in enumerate(model_names):
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        model = get_BF_template(model_name, 
                                    [model_name],
                                    model_name,
                                    data_compression[index], 
                                    ncomponents[index],
                                    offset_types[index],
                                    None,
                                    shift_surprisal[index],
                                    input_template='activations')

        template['models'] = [model]
        template['model_name'] =  'BF_{}_{}'.format(model_name, subject)
        yaml_path = os.path.join(templates_folder, 'BF_{}_{}.yml'.format(model_name, subject))

        save_yaml(template, yaml_path)
        command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
        path_to_sh.append(os.path.join(sh_folder, 'BF_{}_{}.sh'.format(model_name, subject)))


In [57]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    